# Image Segmentation
This section is responsible for detecting the various components of the image such as the lumbar and Spinal Canal

## Processing of Manual Annotation data
Converting the annotation files from `CVAT.xml` to YoloV8 Compatible format

In [2]:
import os
import xml.etree.ElementTree as ET

In [19]:
def convert_cvat_polyline_to_yolo(xml_path, output_dir):
    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Get all unique class names from the polylines
    class_names = sorted(set(
        poly.get("label")
        for image in root.findall("image")
        for poly in image.findall("polygon")
    ))
    class_name_to_id = {name: idx for idx, name in enumerate(class_names)}
    print("Class mapping:", class_name_to_id)

    for image in root.findall("image"):
        img_name = image.get("name")
        img_width = int(image.get("width"))
        img_height = int(image.get("height"))

        label_lines = []
        for poly in image.findall("polygon"):
            label = poly.get("label")
            class_id = class_name_to_id[label]
            point_list = poly.get("points").split(";")

            points = [tuple(map(float, p.split(","))) for p in point_list]
            xs, ys = zip(*points)

            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)

            x_center = (x_min + x_max) / 2 / img_width
            y_center = (y_min + y_max) / 2 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Write YOLO annotation to a text file
        txt_filename = os.path.splitext(img_name)[0] + ".txt"
        with open(os.path.join(output_dir, txt_filename), "w") as f:
            f.write("\n".join(label_lines))
    print(f"files saved successfully")

# Example usage:
xml_input_path = "annotations/annotations_b2.xml"         # Replace with your XML file path
yolo_output_folder = "yolo_labels_b2"         # Replace with your desired output folder
convert_cvat_polyline_to_yolo(xml_input_path, yolo_output_folder)


Class mapping: {'L S': 0, 'SC': 1}
files saved successfully


## Training the YOLO model to recognize the segments in the image

In [33]:
from ultralytics import YOLO

In [34]:
model = YOLO("yolov8n.yaml")

In [35]:
results = model.train(data="config.yaml", epochs=50)

Ultralytics 8.3.47  Python-3.12.7 torch-2.5.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=config.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

train: Scanning C:\Users\RLS\Documents\GitHub\MRIDetection\data\labels\train... 106 images, 0 backgrounds, 0 corrupt: 100%|██████████| 106/106 [00:00<00:00, 1107.61it/s]

train: New cache created: C:\Users\RLS\Documents\GitHub\MRIDetection\data\labels\train.cache



val: Scanning C:\Users\RLS\Documents\GitHub\MRIDetection\data\labels\train.cache... 106 images, 0 backgrounds, 0 corrupt: 100%|██████████| 106/106 [00:00<?, ?it/s]


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50         0G      1.725      1.798       1.86         20        640: 100%|██████████| 7/7 [00:59<00:00,  8.47s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:19<00:00,  4.93s/it]

                   all        106        212      0.802      0.807      0.833      0.422



50 epochs completed in 1.055 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.2MB

Validating runs\detect\train6\weights\best.pt...
WARNING  validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.47  Python-3.12.7 torch-2.5.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
YOLOv8n summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:18<00:00,  4.60s/it]


                   all        106        212      0.797      0.802      0.831      0.422
                    LS        106        106      0.934      0.943      0.974      0.549
                    SC        106        106      0.661      0.661      0.688      0.295
Speed: 3.2ms preprocess, 138.9ms inference, 0.0ms loss, 5.8ms postprocess per image
Results saved to runs\detect\train6


In [36]:
model.names

{0: 'LS', 1: 'SC'}

In [41]:
results2 = model.predict(source='data/images/train', save=True)


image 1/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\ADEBOWALE ADEFISAN.JPG: 640x576 1 LS, 1 SC, 374.8ms
image 2/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH000438.JPG: 512x640 1 LS, 112.3ms
image 3/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH000457.JPG: 640x640 1 LS, 2 SCs, 147.8ms
image 4/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH000633.JPG: 608x640 1 LS, 1 SC, 108.7ms
image 5/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH002542.PNG: 640x608 (no detections), 105.2ms
image 6/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH013809.JPG: 640x640 1 LS, 1 SC, 87.9ms
image 7/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH015875.JPG: 512x640 1 LS, 2 SCs, 83.4ms
image 8/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH015893.PNG: 608x640 1 LS, 94.1ms
image 9/106 C:\Users\RLS\Documents\GitHub\MRIDetection\data\images\train\AH01

In [42]:
newmodel = YOLO("runs/detect/train6/weights/best.pt")
#result2 = newmodel.predict(source="path/to/image.jpg", conf=0.05, save=True)

In [44]:
result2 = newmodel.predict(source="OLD",  imgsz=640)
# print(results[0].boxes.cls)   # See what classes are detected
# print(results[0].boxes.conf)  # See confidence scores


image 1/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data1.jpg: 640x640 1 LS, 311.6ms
image 2/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data10.jpg: 640x640 2 LSs, 1 SC, 115.1ms
image 3/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data2.jpg: 640x640 (no detections), 104.6ms
image 4/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data3.jpg: 640x640 1 SC, 110.2ms
image 5/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data4.jpg: 640x640 2 SCs, 112.3ms
image 6/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data5.jpg: 640x640 (no detections), 107.8ms
image 7/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data6.jpg: 640x640 1 LS, 1 SC, 113.0ms
image 8/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data7.jpg: 640x640 1 LS, 1 SC, 109.1ms
image 9/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data8.jpg: 640x640 (no detections), 98.9ms
image 10/10 C:\Users\RLS\Documents\GitHub\MRIDetection\OLD\data9.jpg: 640x640 1 LS, 115.0ms
Speed: 11.2ms preprocess, 129.8ms i

In [40]:
results2

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'LS', 1: 'SC'}
 obb: None
 orig_img: array([[[  2,   2,   2],
         [  3,   3,   3],
         [  3,   3,   3],
         ...,
         [  3,   3,   3],
         [  3,   3,   3],
         [  3,   3,   3]],
 
        [[  2,   2,   2],
         [  3,   3,   3],
         [  3,   3,   3],
         ...,
         [  3,   3,   3],
         [  3,   3,   3],
         [  3,   3,   3]],
 
        [[  2,   2,   2],
         [  3,   3,   3],
         [  3,   3,   3],
         ...,
         [  3,   3,   3],
         [  3,   3,   3],
         [  3,   3,   3]],
 
        ...,
 
        [[ 76, 186, 182],
         [ 76, 186, 180],
         [ 76, 186, 182],
         ...,
         [ 76, 186, 180],
         [ 76, 186, 182],
         [ 76, 186, 180]],
 
        [[129, 199, 198],
         [129, 199, 198],
         [129, 199, 198],
         ...,
         [129